In [2]:
import quantecon as qe
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy import optimize
import pandas as pd
from scipy.optimize import fsolve

#### Problem Set 5: Solving Aiyagari Model

##### Hasan Cetin, Lucas Belmudes

---

### The Problem

The Bellmann equation of the consumer is the following:

$$ T(V_n)(l,k) = \max_{k'} \left \{ \dfrac{(z-k')^{1-α}}{1-α} + \mathbb{E} \left \{ V_n(l',k') | l \right \} \right \}$$
$$\text{s.t.   } 0 \leq k' \leq z$$
$$\text{where   } z = wl + (1+r)k$$
$$w = (1-θ)\dfrac{K^{α}}{L}, \;\; r = θ \dfrac{K^{-α}}{L} $$

- Assume measure 1 of consumers

- We will assume $l$ is an i.i.d. idiosyncratic process centered at 1. (iid for simplicity, idiosyncracy by Aiyagari Model)

- Thus $L = 1$  and $w,r$ depends only on $K^D$: capital demand.

- $K^S$: capital supply is provided by the consumers/households.

- We will use EGM as in HW4 to solve the consumption-savings problem.

#### The algorithmus

1. Start $K^D$ from $K_{grid}$ and calculate prices $w,r$

2. Solve consumption-savings problem (we used EGM for efficiency) and get $V^*$ and corresponding $Γ^*$

3. Start a simulation for N agents starting from some $k_0$ by creating an $N$ x $T$ shock grid and use $Γ^*$ to simulate their path.
     -  NOTE: Use same shocks for each iteration!!!!!
     -  NOTE: Since this shock grid will use too much RAM size (if T = 20000, N = 1000, then size = 20000 x 1000 x 8 byte = 16 x $10^8$ = 1.6 GB), after calculating path, erase this shock grid and use a seed to create the same shock grid again.

4. $K^S = \dfrac{\sum_{i=1}^{N} k_i^T}{N}$

5. If $K^S > K^D$ increase $K^D$ (i.e. go to next value in the $K_{grid}$ for $K^D$)
     - NOTE: If $K^D > K^S$ even for $K^D = K_{grid}[0]$ then decrease your $K_{grid}$ immediately and start again.

6. Iterate this process until market clearence: $max(|K^S - K^D|) < ϵ$

7. If no convergence and $K^D > K^S$ still, increase your $K_{max}$ for your $K_{grid}$ and continue iteration for $K^D > K_{old max}$ in $K_{newgrid}$

In [3]:
# Define Parameters:

n_k = 500  # Grids for capitals
n_A = 19  # Markov States.
δ = 1     # Full Depreciation.
α = 0.7   # Capital Share.
ρ = 0.95  # Memory of income
σ = 0.2  # Volatility of income.
β = 0.96  # Discont factor.
θ = 1   # Expanding grid coefficient.
error = 10e-6 # Error tolerance.
max_iter = 1000 # Maximum iteration
delta = 10e-4 #To differentiate the value function

#Create aux variables for iteration
dist = 1000
iteration = 0

#Discretizing shocks
markov = qe.markov.approximation.rouwenhorst(n= n_A, ybar = 1-ρ, sigma=σ, rho= ρ)
Π = markov.P   
A = markov.state_values 

# Maximum sustainable Capital:
K_max = 5
K_min = 0.01

# Use and expanding grid for more accuracy:
K_grid = K_min + (K_max - K_min) * (np.linspace(0, 1, n_k)**θ)

In [ ]:
for Kd in K_grid:

    #Calculate the prices:
    w = (1-α)*(Kd**(α))
    r = α * (Kd**(-α))

    #Given Prices, solve the consumption-saving problem:
    #Step 0: Some necessary matrices for EGM.
    V_k = np.zeros((n_A,n_k))
    c_star = np.zeros((n_A,n_k))
    Y_star = np.zeros((n_A,n_k))
    value = np.zeros((n_A,n_k))
    Realvalue = np.zeros((n_A,n_k))

    #Step 1: Creating on grid cash in hands matrix
    Y_prime = np.zeros((n_A,n_k))
    for i in range(n_A):
        for j in range(n_k):
            Y_prime[i,j] = A[i]* (K_grid[j]** α)  + (1-δ)*K_grid[j]

    #Step 2: Initial guess
    V_0 = np.zeros((n_A,n_k))
    for i in range(n_A):
        for j in range(n_k):
            V_0[i,j] = K_grid[j]**(α) * A[i]
    while dist > error and iteration < max_iter:
        for i in range(n_A):
            #Step 3: Doing the numerical derivative of the value matrix wrt k to get c_star
            V_spline = CubicSpline(K_grid,V_0[i,:])
            c_star[i,:] = ((V_spline(K_grid + delta) - V_spline(K_grid - delta))/ (2*delta))**(-1/2)
        
        #Step 4: Finding Y_star
        Y_star = c_star + K_grid
        
        #Step 5: Finding corresponding values
        Values = -c_star **(-1) + V_0
        

        for i in range(n_A):
            Y_i = Y_star[i,:]
            V_i = Values[i,:]

            #We need to sort Y_star to be able to interpolate
            sort_index = np.argsort(Y_i)
            Y_i_sorted = Y_i[sort_index]
            V_i_sorted = V_i[sort_index]

            #Step 6: Interpolate to get on grid values
            interpolation = CubicSpline(Y_i_sorted, V_i_sorted)
            Realvalue[i,:] = interpolation(Y_prime[i,:])

        #Step 7: Update your value
        V_1 = β * np.dot(Π, Realvalue)

        #Step 8: Check your
        dist = np.max(np.abs(V_1 - V_0))
        iteration += 1

        #Step 7: Update your value
        V_0 = V_1.copy()